# Set up a new workflow registry on LifeMonitor

* Registries are the main type of actors allowed to interact with LifeMonitor

* They act with LifeMonitor both as clients and identity providers

* The protocol used to authorize both side of the communication is OAuth2. In more details, we use:
    1. the **authorization code** grant type to authenticate users from workflow registries and authorize LifeMonitor to get their data as needed - actually we get only info about user profiles and workflows.
    2. the **client credentials** grant type to authorize requests from registries

* The reference model of workflow registry we consider is [Seek](https://github.com/seek4science/seek) (aka **WorkflowHub**) which is actually the only supported at the moment. But, in principle, every type of workflow registry which, like WorkflowHub, supports OAuth2 as authorization protocol can be easily integrated in LifeMonitor.

## Registry LifeMonitor as Workflow Registry Client

The first step to set up a new workflow registry is to provide LifeMonitor with the OAuth2 client credentials it needs to access the registry. This implies you need to register LifeMonitor as OAuth2 client application of your registry.
The only info you need from LifeMonitor in this step is the **Redirect URI**, which has always the following form:


      <LIFEMONITOR_BASE_URL>/oauth2/authorized/<REGISTRY_NAME>


... where `REGISTRY_NAME` is the name you'd like to assign to your registry on LifeMonitor.

Taking WorkflowHub as reference, you need to:

1. access the **API Applications** configuration page by clicking on your "user name" and then going through *My Profile > Actions > API Applications > New Application*;
2. choose a name, 
3. set Redirect URI as described above (e.g., https://localhost:8000/oauth2/authorized/seek), 
4. activate Confidential and Scopes > Read, 
5. submit the form.

As a result, you get an **Application UID** and a **Secret** that will be used as **CLIENT_ID** and **CLIENT_SECRET** on the registration process described below.

With respect to the WorkflowHub instance included in dev deployment of LifeMonitor, all the steps described above have already been done and the CLIENT_ID and CLIENT_SECRET you need to complete the registration are:

In [5]:
APP_UID = "ehukdECYQNmXxgJslBqNaJ2J4lPtoX_GADmLNztE8MI"
SECRET = "DuKar5qYdteOrB-eTN4F5qYSp-YrgvAJbz1yMyoVGrk"

## Add the registry to LifeMonitor

LifeMonitor provides an administrative CLI to manage workflow registries which can be accessed from the root path the LifeMonitor folder by typing:

```bash 
flask registry --help
```

If you are using the LifeMonitor `docker-compose` deployment, you can run the command within the `lm` container by typing:

In [28]:
!docker-compose exec lm /bin/bash -c "flask registry --help"

INFO:lifemonitor.app:Logging is active. Log level: INFO
Usage: flask registry [OPTIONS] COMMAND [ARGS]...

Options:
  --help  Show this message and exit.

Commands:
  add     Add a new workflow registry and generate its OAuth2 credentials
  list    List all workflow registries
  show    Show info of a workflow registry
  update  Update a workflow registry




The `registry add` command takes as input the following parameters:

```bash
INFO:lifemonitor.app:Logging is active. Log level: INFO
Usage: flask registry add [OPTIONS] NAME [[seek]] CLIENT_ID CLIENT_SECRET
                          API_URL

  Add a new workflow registry and generate its OAuth2 credentials

Options:
  --redirect-uris TEXT            Redirect URIs (comma separated) to be used
                                  with authorization code flow

  --client-auth-method [client_secret_basic|client_secret_post]
                                  Specifies the method used for authenticating
                                  the registry with LifeMonitor

  --help                          Show this message and exit.
```

With respect to the `seek` registry instance we are considering, they should be set as follows:

In [8]:
NAME = "seek"                             # the name to be assigned to the registry on LifeMonitor
TYPE = "seek"                             # the registry type ('seek' is the only value allowed at the moment)
CLIENT_ID = APP_UID                       # LifeMonitor APP_UID from WorkflowHub
CLIENT_SECRET = SECRET                    # LifeMonitor SECRET from WorkflowHub
CLIENT_AUTH_METHOD="client_secret_post"   # OAuth2 authentication method (client_secret_post|client_secret_basic) 
API_URL="https://seek:3000"               # URL of the workflow registry
REDIRECT_URIS="https://seek:3000"         # URL of the OAuth2 callback on the workflow registry

You can now easily add your registry on LifeMonitor through the following command (assumed to be launched from the root of the LifeMonitor repository):

In [18]:
add_registry_cmd = f"""
docker-compose exec lm /bin/bash -c \
"flask registry add {NAME} {TYPE} {CLIENT_ID} {CLIENT_SECRET} {API_URL} \
--client-auth-method {CLIENT_AUTH_METHOD} \
--redirect-uris {REDIRECT_URIS}"
"""
print(add_registry_cmd)


docker-compose exec lm /bin/bash -c "flask registry add seek seek ehukdECYQNmXxgJslBqNaJ2J4lPtoX_GADmLNztE8MI DuKar5qYdteOrB-eTN4F5qYSp-YrgvAJbz1yMyoVGrk https://seek:3000 --client-auth-method client_secret_post --redirect-uris https://seek:3000"



In [27]:
!{add_registry_cmd}

INFO:lifemonitor.app:Logging is active. Log level: INFO
INFO:lifemonitor.commands.registry:Registry 'seek' created!


****************************************************************************************************
Workflow Registry 'seek' (uuid: 1c40002b-8676-4f51-b529-9ff02ff78234, type: seek) registered!
****************************************************************************************************


OAuth2 settings to connect to LifeMonitor:
----------------------------------------------------------------------------------------------------
REGISTRY NAME: seek
REGISTRY API URL: https://seek:3000
REGISTRY CLIENT ID: bAVlsOwx2Z7sPgQJU7mdg1CE
REGISTRY CLIENT SECRET: FQ8FE57DvYy9C5NKsAgXaBXAQ061gsNXz71X8I8radX6zHU2
REGISTRY CLIENT ALLOWED SCOPES: read write
REGISTRY CLIENT ALLOWED FLOWS: ['client_credentials', 'authorization_code', 'refresh_token']
REGISTRY CLIENT REDIRECT URIs: ['https://seek:3000']
REGISTRY CLIENT AUTH METHOD: client_secret_post
AUTHORIZE URL: <LIFE_MONITOR_

The parameters above allows you to set up a LifeMonitor client on your workflow registry.